In [1]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from urllib.parse import urlparse
import os
import numpy as np
from IPython.display import clear_output, display
import time
import pandas as pd
from Constants import Constants, specialTokenList, specialTokens
from All_Models import SSCL, GatedCNN, SelfAttnModel
from utils import getSampler
from LoadData import loadingTweetsAndUserInfoData
import torch
from collections import defaultdict

from torch.utils.data import Dataset, DataLoader


%matplotlib inline


'''
TODO:

1. Mardularize 
2. Pickle the output
3. Training


'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




#Using inplace method for the dat

class args(object):

    # Data
    
    
    dataset = "HSpam14"
    full_data = True
    usingWeightRandomSampling = True
    vocab_size = 8000 # if we create the new vocab size, we have to do the new preprocess again
    validation_portion = 0.05
    test_portion = 0.04
    random_seed = 64
    
    pickle_name = "UserAndTweetsFullPickleData"+ str(vocab_size) + "Vocab.txt"
    pickle_name_beforeMapToIdx = "UserAndTweetsFullPickleDatabeforeMapToIdx.txt"

    

    

    ##### Arch

    ## GatedCNN arch

    GatedCNN_embedingDim = 128
    GatedCNN_convDim = 64
    GatedCNN_kernel = 3
    GatedCNN_stride = 1
    GatedCNN_pad = 1
    GatedCNN_layers = 8
    GatedCNN_dropout = 0.1
        
    ## SSCL arch
    
    
    SSCL_embedingDim = 512
    SSCL_RNNHidden = 256
    SSCL_CNNDim = 256
    SSCL_CNNKernel = 5
    SSCL_CNNDropout = 0.1
    SSCL_LSTMDropout = 0.1
    SSCL_LSTMLayers = 1
    
    ## Attn arch

    SelfAttn_LenMaxSeq = 280 # Default, will be changed Later

    # These Two has to be the same
    SelfAttn_WordVecDim = 128
    SelfAttn_ModelDim = 128
    
    SelfAttn_FFInnerDim = 256
    SelfAttn_NumLayers = 3
    SelfAttn_NumHead = 4
    SelfAttn_KDim = 64
    SelfAttn_VDim = 64
    SelfAttn_Dropout = 0.1
    
    
    ## MultiTask Model
    
    FC_hidden = 16
    
    
    # Training params
    
    batch_size = 64
    L2 = 0.1
    threshold = 0.5
    lr = 0.002
    n_epoch = 50

    # If using Adam
    adam_beta1 = 0.9
    adam_beta2 = 0.999
    
    earlyStopStep = 2000 # Set None if we don't want it
    earlyStopEpoch = 1 #

    # Logging the Training
    val_freq = 50
    val_steps = 3
    log_freq = 10
    model_save_freq = 1
    model_name = 'TestingModel'
    model_path = './'+ dataset +'_Log/' + model_name + '/Model/'
    log_path = './' + dataset +'_Log/' + model_name + '/Log/'
    
args.device = device

# Create the path for saving model and the log
if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)

if not os.path.exists(args.log_path):
    os.makedirs(args.log_path)

In [7]:
len(df)

1145136

In [10]:
df

,text,numberOfHashtags_c,favorite_count,retweet_count,maliciousMark,possibly_sensitive,followers_count,friends_count,default_profile,default_profile_image,favourites_count,listed_count,statuses_count,verified
0,"She's back climbing - if you love it, maybe yo...",0,0,0,0,0.0,13081,741,0,0,3063,306,6963,0
1,Sweet victory. #warriors http://t.co/d8LnaVwphD,1,1,1,0,0.0,20988,1774,0,0,21909,604,14678,0
2,Vegetable bounty from the Barrel house in Muir...,0,0,0,0,0.0,3942,84,0,0,4,115,1961,0
3,@superfoot just ridiculous. The Knicks are the...,0,0,0,0,NaN,1170932,420,0,0,7105,2538,18662,1
4,@shroo i'm thinking black matte glitter on all...,0,0,0,0,NaN,3340,427,0,0,4892,305,28058,0
5,".@CharlesCMann btw, here you are in my Brain: ...",0,1,0,0,0.0,9605,1883,0,0,742,731,20920,0
6,WHY DOES NEW @GMAIL COMPOSE SUCK SO MUCH?! I w...,0,0,1,0,NaN,12464,1530,0,0,6708,496,9030,0
7,"@David__John oh, I love it!! Want those in the...",0,0,0,0,NaN,2775,783,0,0,37184,103,47286,0
8,RT @bpbdelhi: This just in: we hear the excise...,0,0,17,0,NaN,21845,1125,0,0,4868,746,149732,1
9,In testing Android for @Circa inspiration I've...,0,0,1,0,NaN,24790,737,0,0,4466,1478,30280,1
